In [49]:
import pyaudio
import sounddevice as sd
import wavio
import numpy as np 
import keyboard  

In [50]:
p = pyaudio.PyAudio()

device_count = p.get_device_count()

built_in_mic = None

# Check if it's an input device (microphone)
        # Check if "built-in" is in the device name
for i in range(device_count):
    device_info = p.get_device_info_by_index(i)
    if device_info['maxInputChannels'] > 0:  
        if "built-in" in device_info['name'].lower():
            built_in_mic = i
            print(f"Built-in Microphone found: Device {i} - {device_info['name']}")

if built_in_mic is None:
    print("No built-in microphone found.")


No built-in microphone found.


In [51]:
# Loop through the devices and print their names and indices

for i in range(device_count):
    device_info = p.get_device_info_by_index(i)
    if device_info['maxInputChannels'] > 0: 
        print(f"Device {i}: {device_info['name']}")

# Terminate the PyAudio instance
p.terminate()

Device 0: Microsoft Sound Mapper - Input
Device 1: Headset (soundcore P20i Hands-F
Device 6: Primary Sound Capture Driver
Device 7: Headset (soundcore P20i Hands-Free AG Audio)
Device 15: Headset (soundcore P20i Hands-Free AG Audio)
Device 21: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(soundcore P20i))
Device 23: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(M90max))
Device 25: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(GL-HP201))


In [52]:
from threading import Event

def record_audio():
    sample_rate = 44100
    stop_event = Event()  # to stop recording

    print("Recording... Press 'Enter' to stop.")

    # Start recording in a non-blocking way
    recording = []
    def callback(indata, frames, time, status):
        recording.append(indata.copy())
        if stop_event.is_set():  
            raise sd.CallbackStop

    # Begin recording with callback
    stream = sd.InputStream(
        samplerate=sample_rate,
        channels=1,
        dtype='int16',
        device=1,
        callback=callback
    )
    stream.start()

    # Wait for Enter key to stop
    keyboard.wait('enter')
    stop_event.set()  
    stream.stop()
    stream.close()

    # Save the recording
    if recording:
        recording = np.concatenate(recording)
        audio_file_path = "recorded_audio.wav"  
        wavio.write(audio_file_path, recording, sample_rate, sampwidth=2)
        print(f"Audio saved as '{audio_file_path}'")
        return audio_file_path  # Return the path

if __name__ == "__main__":
    audio_path = record_audio()  

Recording... Press 'Enter' to stop.
Audio saved as 'recorded_audio.wav'
